# Resumo do Código

Cria um mapa interativo dos setores de coleta de lixo, mostrando:
- **Setores de coleta**: Coloridos por horário (diurno/noturno) e dias da semana
- **Pontos de descarte**: Marcadores diferenciando dados coletados (azul) e estimados (vermelho)
- **Camadas interativas**: Permite alternar entre visualizações de horário, dias de coleta e pontos de descarte
- **Tooltips informativos**: Exibe detalhes como setor, horário e frequência de coleta ao passar o mouse

**Objetivo**: Visualização espacial da operação de coleta de resíduos e pontos de descarte irregular na cidade.

In [1]:
import geopandas as gpd
import pandas as pd
import os
import json
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.features import GeoJson, GeoJsonTooltip

import warnings
warnings.filterwarnings(
    "ignore",
    message=".*choropleth_mapbox.*deprecated.*")


In [2]:
# leitura da geometria de coleta
filename="shape_coleta.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo = os.path.join(path, filename)

gdf = gpd.read_file(arquivo)

gdf['num_dias'] = gdf['dias'].apply(lambda x: len(eval(x)))
gdf=gdf[['area_coleta',	'setor','horario',	'dias',	'num_dias','latitude_ref','longitude_ref',	'geometry']]

# leitura dos pontos de descartes
filename2="Pontos_descartes_ML.gpkg"
path='/home/akel/PycharmProjects/Data_ambiental/data/process/'
arquivo2 = os.path.join(path, filename2)
gpd_p = gpd.read_file(arquivo2)

In [3]:
# -----------------------------------------------------
# CONFIGURAÇÕES
# -----------------------------------------------------
centro_lat, centro_lon = -1.427897, -48.4162631

gdf_simplified = gdf.copy()

# Paleta de cores original
color_schemes = {
    'horario': {
        'diurno': {'fill': 'rgba(0, 255, 0, 0.4)', 'line': 'green'},
        'noturno': {'fill': 'rgba(0, 0, 139, 0.4)', 'line': 'darkblue'}
    },
    'num_dias': {
        3: {'fill': 'rgba(255, 165, 0, 0.4)', 'line': 'orange'},
        6: {'fill': 'rgba(0, 255, 0, 0.4)', 'line': 'green'},
        'default': {'fill': 'rgba(100, 100, 100, 0.4)', 'line': 'gray'}
    }
}

# -----------------------------------------------------
# CORES PARA OS PONTOS (gpd_p)
# -----------------------------------------------------
ponto_colors = {
    "Estimados": "red",
    "Dados": "darkblue"}

# -----------------------------------------------------
# INICIAR MAPA
# -----------------------------------------------------
mapa_coleta = folium.Map(
    location=[centro_lat, centro_lon],
    zoom_start=13,
    tiles="CartoDB Positron",
    control_scale=True)

# -----------------------------------------------------
# CAMADA 1 — HORÁRIO
# -----------------------------------------------------
def style_horario(feature):
    h = feature["properties"]["horario"]
    c = color_schemes["horario"].get(h, {'fill': "rgba(100,100,100,0.4)", "line": "gray"})
    return {
        "fillColor": c["fill"],
        "color": c["line"],
        "weight": 1,
        "fillOpacity": 0.6 }

layer_horario = folium.FeatureGroup(name="Horário", show=True)

folium.GeoJson(
    data=json.loads(gdf_simplified.to_json()),
    name="Horário",
    style_function=style_horario,
    tooltip=GeoJsonTooltip(
        fields=["setor", "horario", "num_dias"],
        aliases=["Setor", "Horário", "Dias/Semana"],
        sticky=True)
).add_to(layer_horario)

layer_horario.add_to(mapa_coleta)

# -----------------------------------------------------
# CAMADA 2 — DIAS DA SEMANA
# -----------------------------------------------------
def style_dias(feature):
    nd = feature["properties"]["num_dias"]
    c = color_schemes["num_dias"].get(nd, color_schemes["num_dias"]["default"])
    return {
        "fillColor": c["fill"],
        "color": c["line"],
        "weight": 1,
        "fillOpacity": 0.6}

layer_dias = folium.FeatureGroup(name="Dias/Semana", show=False)

folium.GeoJson(
    data=json.loads(gdf_simplified.to_json()),
    name="Dias/Semana",
    style_function=style_dias,
    tooltip=GeoJsonTooltip(
        fields=["setor", "horario", "num_dias"],
        aliases=["Setor", "Horário", "Dias/Semana"],
        sticky=True)).add_to(layer_dias)

layer_dias.add_to(mapa_coleta)

# -----------------------------------------------------
# CAMADA 3 — PONTOS (Estimados / Dados)
# -----------------------------------------------------
layer_pontos = folium.FeatureGroup(name="Pontos (Estimados/Dados)", show=True)

for _, row in gpd_p.iterrows():
    bairro = row["Bairro"]
    lat = row["lat"]
    lon = row["lon"]
    cor = row["Cor"]  # Estimados ou Dados
    
    marker_color = ponto_colors.get(cor, "black")  # fallback
    
    popup_html = f"""
    <b>Bairro:</b> {bairro}<br>
    <b>Tipo:</b> {cor}<br>
    <b>Lat:</b> {lat:.5f}<br>
    <b>Lon:</b> {lon:.5f}
    """

    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.9,
        popup=popup_html,
        tooltip=f"{bairro} — {cor}"
    ).add_to(layer_pontos)

layer_pontos.add_to(mapa_coleta)


folium.LayerControl(collapsed=False).add_to(mapa_coleta)

mapa_coleta